In [1]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import nbimporter

In [23]:
from ResNet34 import ResnetBuilder
from Evaluate import get_model
from keras.utils import Sequence
from DataParserForCNN import ImageBatchGenerator
from DataParserForCNN import read_class, read_full_df, null_augment, drawing_to_image

In [3]:
def normalize(image):
    return image / 255.

In [4]:
model = get_model(net='resnet34', weight_path='./model/quick_draw_resnet34_1016.h5')

In [5]:
base_path = '/Volumes/JS/QuickDraw/'
class_array = read_class()

In [7]:
full_df, train_id, valid_id = read_full_df(base_path, class_array)

	 load df   :  339/340                   zigzag   7 min 26 sec

	 load valid_set split:  339/340                   zigzag  11 min 06 sec

In [27]:
train_batch = ImageBatchGenerator(base_path=base_path,
                                  class_array=class_array,
                                  full_df=full_df,
                                  datasplit_id=train_id,
                                  split='train_set',
                                  batch_size=2048, 
                                  augment=null_augment, 
                                  mode='simplified', 
                                  norm=normalize,
                                  shuffle=True)


In [64]:
class TestGenerator(Sequence):
    def __init__(self, 
                 test_df,
                 datasplit_id,
                 split='test_set',
                 augment=null_augment, 
                 batch_size=128,
                 mode='simplified',
                 norm=None,
                 shuffle=True):
        """ data: could be 'train' or 'test', the former used to get splited dataset train_set or valid_set, 
                  the latter used to get the TEST_DF
        """
        assert mode in ['simplified', 'raw']
        
        self.datasplit_id = datasplit_id
        self.split = split
        self.norm = norm
        self.df = test_df
        
        self.augment = augment
        self.batch_size = batch_size
        self.mode = mode 
        self.H, self.W = 64, 64

        num_class = 340

        print('')

        ### shuffle                    
        if shuffle:
            np.random.shuffle(self.datasplit_id)
        print('\n')

        
    def __str__(self):
        N = len(self.datasplit_id)
        string = ''\
        + '\tdata         = %s\n'%self.split \
        + '\tmode         = %s\n'%self.mode \
        + '\tlen(self.datasplit_id) = %d\n'%N \
        + '\n'
        return string


    def __getitem__(self, idx):
        if self.split != 'test_set':
            l_bound = idx * self.batch_size
            r_bound = (idx + 1) * self.batch_size
            if r_bound > len(self.datasplit_id):
                r_bound = len(self.datasplit_id)
                l_bound = r_bound - self.batch_size
            instance_count = 0

            x_batch = np.zeros((r_bound-l_bound, self.H, self.W, 3))
            y_batch = np.zeros((r_bound-l_bound, 340), dtype='uint32')

            for instance in self.datasplit_id[l_bound:r_bound]:
                label, drawing_id, key_id = instance
                drawing = self.full_df[label]['drawing'][drawing_id]
                drawing = eval(drawing)

                image, label, cache = self.augment(drawing, label, idx)

                x_batch[instance_count] = image
                y_batch[instance_count, int(label)] = 1

                if self.norm != None:
                    x_batch[instance_count] = self.norm(image)

                instance_count += 1
                
            return [x_batch, y_batch]
            
        else:
            l_bound = idx * self.batch_size
            r_bound = (idx + 1) * self.batch_size
            
            if r_bound > len(self.datasplit_id):
                r_bound = len(self.datasplit_id)
                l_bound = r_bound - self.batch_size
            instance_count = 0

            x_batch = np.zeros((r_bound - l_bound, self.H, self.W, 3))
            
            for instance in self.datasplit_id[l_bound:r_bound]:
                drawing_id = instance
                drawing = self.df['drawing'][idx]
                print(drawing)
                print(type(drawing))
                drawing = eval(drawing)
                
                label = None
                image, label, cache = self.augment(drawing, label, idx)
                
                x_batch[instance_count] = image
                
                if self.norm != None:
                    x_batch[instance_count] = self.norm(image)

                instance_count += 1

            return x_batch


    def __len__(self):
        return int(np.ceil(len(self.datasplit_id) / self.batch_size))

In [50]:
valid_df = []
for i in range(len(full_df)):
    print(i)
    valid_df.append(full_df[i])
valid_df = pd.concat(valid_df)

340
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

In [65]:
valid_batch = TestGenerator(test_df=valid_df,
                            datasplit_id=valid_id,
                            split='test_set',
                            batch_size=1, 
                            augment=null_augment, 
                            mode='simplified', 
                            norm=normalize,
                            shuffle=False)

In [66]:
valid_batch[0]

0    [[[0, 22, 37, 64, 255], [218, 220, 227, 228, 2...
0    [[[10, 41, 126, 153, 195, 219, 241], [49, 42, ...
0    [[[101, 98, 91, 83, 80, 80, 82, 92, 104, 117, ...
0    [[[167, 109, 80, 69, 58, 31, 57, 117, 99, 52, ...
0    [[[111, 148, 161, 175, 199, 218, 231, 236, 234...
0    [[[13, 10, 12, 20, 39, 67, 170, 196, 213, 220,...
0    [[[93, 103, 104, 139, 140, 149, 84], [174, 111...
0    [[[19, 52], [44, 50]], [[0, 0, 5], [56, 65, 69...
0    [[[27, 17, 16, 21, 34, 50, 49, 34, 23, 17], [4...
0    [[[50, 107, 188, 198, 201, 200, 195, 156, 147,...
0    [[[255, 255], [0, 0]], [[255, 255], [0, 0]], [...
0    [[[6, 13, 41, 56, 66, 95, 118, 129, 152, 190, ...
0    [[[0, 51, 62, 100], [255, 129, 111, 67]], [[8,...
0    [[[2, 38, 71, 108, 128, 139], [10, 24, 31, 22,...
0    [[[96, 67, 48, 46, 53, 69, 87, 132, 164, 191, ...
0    [[[4, 18, 29, 63, 93, 120, 146, 169, 186, 218,...
0    [[[61, 31, 4, 0, 0, 19, 33, 193, 216, 234, 255...
0    [[[35, 28, 7, 3, 108, 163, 180, 196, 229, 241,...
0    [[[13

TypeError: eval() arg 1 must be a string, bytes or code object

In [32]:
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy


def top_3(y_true, y_pred): 
    return top_k_categorical_accuracy(y_true, y_pred, 3)

def top_1(y_true, y_pred): 
    return top_k_categorical_accuracy(y_true, y_pred, 1)

def map_3_keras(y_true, y_pred):
    return (top_k_categorical_accuracy(y_true, y_pred, 3) + top_k_categorical_accuracy(y_true, y_pred, 2)
            + top_k_categorical_accuracy(y_true, y_pred, 1)) / 3

def ap_3(y_true, y_pred):
    score = 0.0
    num_tp = 0.0
    
    if len(y_pred) > 3:
        y_pred = y_pred[:3]

    for i, p in enumerate(y_pred):
        if p in y_true and p not in y_pred[:i]:
            num_tp += 1.0
            score += num_tp / (i+1.0)

    if not y_truth:
        return 0.0
    
    return score / min(len(y_true), 3)


def map_3(y_true, y_pred):
    """ map average precision with maximum 3 predicted elements """
    return np.mean([ap_3(a, p) for a, p in zip(y_true, y_pred)])


optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', 
              metrics = ['accuracy', top_1, top_3, map_3_keras],
              optimizer=optimizer)

In [33]:
result = model.evaluate_generator(valid_batch, verbose=1)

213/213 [==============================] - 268s 1s/step


In [42]:
print('top_1 %2.3f%%, top_3 %2.3f%%, MAP@3 %2.3f%%' % (100*result[2], 100*result[3], 100*result[4]))

top_1 81.030%, top_3 92.998%, MAP@3 88.095%


In [44]:
from sklearn.metrics import confusion_matrix, classification_report

In [51]:
print(valid_batch[0][1][0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]


In [59]:
valid_y = [np.argmax(valid_batch[i][1][j]) for i in range(0, len(valid_batch))]

In [65]:
np.shape(valid_y)

(213,)

In [77]:
len(valid_batch[0][1][0])

340

In [82]:
valid_y_pred = []
for i in range(len(valid_batch)):
    cur_batch = [np.argmax(valid_batch[i][1][j]) for j in range(128)]
    valid_y_pred.append(cur_batch)
    print(valid_y_pred)

[[115, 321, 229, 71, 53, 186, 99, 166, 333, 272, 291, 89, 229, 104, 323, 117, 155, 131, 326, 223, 27, 76, 30, 311, 155, 180, 306, 88, 226, 82, 235, 302, 133, 181, 160, 238, 267, 274, 19, 168, 278, 3, 321, 226, 223, 32, 234, 196, 262, 236, 104, 179, 170, 187, 161, 317, 198, 227, 308, 69, 262, 284, 10, 231, 10, 226, 96, 321, 234, 5, 298, 75, 42, 169, 261, 312, 322, 211, 188, 250, 205, 308, 34, 313, 242, 231, 212, 115, 228, 191, 6, 83, 29, 120, 169, 297, 103, 278, 94, 222, 339, 308, 62, 16, 281, 62, 5, 329, 198, 255, 194, 87, 105, 84, 122, 25, 297, 23, 143, 24, 52, 267, 314, 185, 108, 54, 147, 323]]
[[115, 321, 229, 71, 53, 186, 99, 166, 333, 272, 291, 89, 229, 104, 323, 117, 155, 131, 326, 223, 27, 76, 30, 311, 155, 180, 306, 88, 226, 82, 235, 302, 133, 181, 160, 238, 267, 274, 19, 168, 278, 3, 321, 226, 223, 32, 234, 196, 262, 236, 104, 179, 170, 187, 161, 317, 198, 227, 308, 69, 262, 284, 10, 231, 10, 226, 96, 321, 234, 5, 298, 75, 42, 169, 261, 312, 322, 211, 188, 250, 205, 308, 34, 3

KeyboardInterrupt: 

In [95]:
valid_batch[0]

[array([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],
 
 
        [

In [12]:
valid_y_preds = model.predict_generator(valid_batch, verbose=1)

StopIteration: 'TestGenerator' object has no attribute 'full_df'